In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')

## Billionaires

Let's do some examples with exploratory analysis of a dataset, to give you an idea of where you might start if you are handed a dataset and want to get a sense of what is in there.  Here we look at a dataset of billionaires around the world.

In [ ]:
billionaires = Table.read_table('billionaires.csv')
billionaires

Maybe it would be interesting to understand where around the world billionaires are located.  What visualization would be most suitable?

In [ ]:
by_country = billionaires.group('citizenship')
by_country.sort(1, descending=True).barh('citizenship')

We can also break things down by industry sector.

In [ ]:
by_industry = billionaires.group('industry')
by_industry.sort(1, descending=True).barh('industry')

It occurred to us that maybe the industry segment with the billionaires depends on the country.  For instance, maybe in the US there are more billionaires in the tech sector, but maybe in some other countries there are more billionaires in manufacturing.  Let's check that out.

In [ ]:
billionaires.pivot('industry', 'citizenship').show()

That's unwieldy, because there are so many countries.  It would be clearer if we could break this down by continent.  So, let's find a dataset that has that mapping, and then hook it up to the billionaires dataset.

In [ ]:
countries = Table.read_table('countries.csv')
countries

In [ ]:
countries.where('Continent', 'North America')

In [ ]:
billionaires = billionaires.join('citizenship', countries, 'Country')
billionaires

In [ ]:
billionaires.pivot('industry', 'Continent')

We got curious about political donations coming from the very rich, so here is a dataset of donations to political candidates.  Here is a dataset of donations from CEOs.  In this dataset, donations to Democratic candidates are coded as party = 100, and Republican corresponds to party = 200.

In [ ]:
ceos = Table.read_table('ceo.csv')
ceos

In [ ]:
ceos.hist('amount')

Which CEOs have donated the most amount?  Do you recognize any of the names at the top?

Incidentally, check out the 3rd and 4th row.  A common problem in data science is that often the data is a little messy, and before you do an analysis you may need to clean up the data, or otherwise your conclusions might be faulty.  In this class we mostly give you datasets where we have already cleaned the data, to spare you from the mess, but in practice it's often a big deal.

In [ ]:
ceos.sort('amount', descending=True)

In [ ]:
ceos.sort('amount', descending=False)

In [ ]:
ceos.hist('amount', bins=np.arange(0, 100000, 1000), unit='$1000')

We can match this up with the billionaires dataset we were looking at before, to find out information about CEOs who are also billionaires.

In [ ]:
b_donations = billionaires.join('name', ceos, 'ceo')
b_donations.show()

In [ ]:
b_donations.where('amount', are.below(6000000)).scatter('net worth', 'amount')

In [ ]:
donation_prop = b_donations.column('amount') / b_donations.column('net worth')
b_donations = b_donations.with_column('Contribution%', donation_prop)
b_donations = b_donations.sort('amount', descending=True)
b_donations.set_format('Contribution%', PercentFormatter)
b_donations.show(20)

In [ ]:
.02 / 100 * 20000

## Meteorites

Let's look at a dataset with information about various meteorites.  We can again practice some exploratory data analysis to understand what's out there.

In [ ]:
meteorites = Table.read_table('meteorites_na.csv')
meteorites

In [ ]:
meteorites.group('class').sort('count', descending=True)

In [ ]:
meteorites.group('how')

In [ ]:
meteorites.group('how', np.average)

In [ ]:
meteorites.hist('year', bins=np.arange(1800, 2020, 5), group='how')

In [ ]:
meteorites.scatter('year', 'mass')

In [ ]:
meteorites.where('mass', are.below(10000)).scatter('year', 'mass')

In [ ]:
new_fell = meteorites.where('how', 'Fell').where('year', are.above(1950))
new_found  = meteorites.where('how', 'Found').where('year', are.above(1950))
new_found

## Maps

Let's see a new type of visualization: a map.  The 

In [ ]:
Marker.map_table(new_fell.select('lat', 'lon', 'name'))

In [ ]:
Circle.map_table(new_found.select('lat', 'lon', 'name'), radius=10)

## Bikes

Let's now look at a dataset of bicycle trips in the Bay area, which was released publicly about a bike sharing system where people could rent a bike from any of a number of locations and then take them to another location.

If you loaded the meteorites dataset above, now might be a good time to restart your kernel (Kernel > Restart and clear output), run the cell at the top of the notebook, then come back here. 

The bike datasets in this section is big, and could lead to kernel crashes.

The following table has one row per trip taken, showing where and when it was checked out and where and when it was returned.

In [ ]:
trip = Table.read_table('trip.csv')
trip

Maybe one of the first things that might be interesting to look at in would be the duration of the bike trips.

In [ ]:
trip.hist('Duration')

Why does the histogram look so weird?  Let's take a closer look:

In [ ]:
commute.sort('Duration', descending=True)

What's going on with the top row?  That bike was stolen!  That's not really what we are interested in.  As it happens, the service was free for commuters as long as their trip took less than 30 minutes, so let's focus only on the trips that are likely to represent those kinds of commuter use.

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.hist('Duration', bins=60, unit='second')

In [ ]:
# Percent of people who have a ride duration between 500 and 250 seconds
(500-250) * 0.15 

In [ ]:
starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
commute.pivot('Start Station', 'End Station')

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
shortest = duration.group(['Start Station', 'End Station'], min).relabeled('Duration min', 'Minimum Duration')
shortest

In [ ]:
from_cc = shortest.where('Start Station', are.containing('Civic Center BART')).sort('Minimum Duration')
from_cc

## Maps, again

In [ ]:
stations = Table.read_table('station.csv')
stations

In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name'))

In [ ]:
sf = stations.where('landmark', 'San Francisco')
Circle.map_table(sf.select('lat', 'long', 'name'), color='green', radius=15)

## Extra practice

### Write a Python expression below each of the following descriptions that computes its value. You may use more than one line.

In [ ]:
# The name of the station where the most rentals ended (assume no ties).

In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.

In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends

In [ ]:
# The name of the station where the most rentals ended (assume no ties).
# First, find end counts
# Then, find the station with the highest end count
trip.group('End').sort('count', descending=True).column(0).item(0)

In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.

# First, find the average end time for each station
# Then, keep the ones above 10 minutes
# Then, count them
trip.group('End', np.average).where(2, are.above(10*60)).num_rows

In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends.

# First, group the trip on starts and ends, separately
# Then, join the two grouped tables
# Then, count the number of rows where the stations' start and end counts are bove 500
start_counts, end_counts = trip.group('Start').relabeled('count', 'Start Count'), trip.group('End').relabeled('count', 'End Count')
start_counts.join('Start', end_counts, 'End').where(
    'Start Count', are.above(500)).where('End Count', are.above(500)).num_rows